In [0]:
from pyspark.sql.functions import sum,count,when,col

In [0]:
%run "/Workspace/Projects/formula1/Data Ingestion/Prod/01-setup"

In [0]:
race_result_df=spark.read.parquet(f"{output_folder_path}/final")

In [0]:
driver_standings_df= race_result_df.groupBy("race_year","driver_name","driver_nationality")\
                              .agg(sum("points").alias("tot_points"), count(when(col("position")==1, True)).alias("wins"))
                                  

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank, asc

driver_rank_spec= Window.partitionBy("race_year").orderBy(desc("tot_points"), desc("wins"))
final_df=driver_standings_df.withColumn("rank", rank().over(driver_rank_spec))

In [0]:
final_df.write.mode("overwrite").saveAsTable("f1_processed.calc_race_results")